<a href="https://colab.research.google.com/github/usshaa/Cheatsheets/blob/main/Notes_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required libraries


In [12]:
!pip install gradio==3.50.0
!pip install google-generativeai
!pip install reportlab
!pip install pillow

# Importing required libraries

In [13]:
import gradio as gr
import google.generativeai as genai
import io
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from google.colab import userdata

# Setup API Key to access Gemini LLM

In [14]:
# Access your API key from Colab Secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Text Generation model declaration from google gemini model

In [21]:
# Use only the text model
text_model = genai.GenerativeModel('gemini-1.5-flash')

# Function to set styles for pdf file

In [16]:
def create_pdf(content, filename="educational_content.pdf"):
    """Creates a PDF from the generated text content."""
    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter
    textobject = c.beginText(50, height - 50)
    textobject.setFont("Times-Roman", 12)

    lines = content.split('\n')
    y_position = height - 50

    for line in lines:
        textobject.textLine(line)
        y_position -= 14
        if y_position < 50:
            c.drawText(textobject)
            c.showPage()
            textobject = c.beginText(50, height - 50)
            textobject.setFont("Times-Roman", 12)
            y_position = height - 50

    c.drawText(textobject)
    c.save()
    buffer.seek(0)
    return buffer.getvalue()

# Function to fine tune the model for content generation

In [17]:
def generate_content(topic, audience, format_type, complexity):
    """Generates educational content using Gemini."""
    prompt = f"""
    You are an AI assistant specializing in creating educational content for educators.
    Generate comprehensive study notes for students based on the following criteria, suitable for exam preparation:

    Topic: {topic}
    Target Audience: {audience} (e.g., High School Students, College Undergraduates)
    Complexity Level: {complexity} (e.g., Beginner, Intermediate, Advanced)

    The content should be detailed, accurate, and relevant to the specified topic and audience.
    Include key concepts, definitions, explanations, and examples that students can use to study for exams.
    The format should be structured as study notes, not a lesson plan or outline.
    """

    try:
        response = text_model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"An error occurred during text generation: {e}"

# Function to create pdf file with generated response

In [18]:
def process_and_create_pdf(topic, audience, format_type, complexity):
    """Generates detailed content with explanation using some examples and creates a downloadable PDF."""
    generated_text = generate_content(topic, audience, format_type, complexity)
    pdf_data = create_pdf(generated_text)

    temp_filename = "generated_content.pdf"
    with open(temp_filename, "wb") as f:
        f.write(pdf_data)

    return generated_text, temp_filename

# Build UI with Gradio

In [19]:
# Gradio UI
iface = gr.Interface(
    fn=process_and_create_pdf,
    inputs=[
        gr.Textbox(label="Topic"),
        gr.Textbox(label="Target Audience"),
        gr.Textbox(label="Format (e.g., Lesson Plan, Lecture Notes, Quiz Questions)"),
        gr.Textbox(label="Complexity Level (e.g., Beginner, Intermediate, Advanced)")
    ],
    outputs=[
        gr.Markdown(label="Generated Content"),
        gr.File(label="Download PDF")
    ],
    title="Educator Content Creation",
    description="Generate educational content and download it as a PDF."
)

# Run the app

In [20]:
iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c97876b5e4735be3d7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c97876b5e4735be3d7.gradio.live
